In [ ]:
import pandas as pd

import numpy as np

#from numba import njit

# import matplotlib

import cudf
print("cudf version:", cudf.__version__)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
# # slow
# train = pd.read_csv('../input/jane-street-market-prediction/train.csv')

In [ ]:
# fast
# %%time
train_cudf = cudf.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
train = train_cudf.to_pandas()
del train_cudf

In [ ]:
train.dtypes

In [ ]:
end_memory = train.memory_usage().sum() / 1024**2
print(f"Memory usage of dataframe {end_memory} MB")

In [ ]:
train.loc[:,'weight':'feature_129'] = train.loc[:,'weight':'feature_129'].astype('float16')

In [ ]:
end_memory = train.memory_usage().sum() / 1024**2
print(f"Memory usage of dataframe {end_memory} MB")

In [ ]:
features = pd.read_csv('../input/jane-street-market-prediction/features.csv')
example_test = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')
sample_prediction_df = pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv')
print ("Data is loaded!")

In [ ]:
#train = train[train['weight'] != 0]

train['action'] = (train['resp'].values > 0).astype('int')


x_train = train.loc[:, train.columns.str.contains('feature')]
y_train = train.loc[:, 'action']

In [ ]:
x_train = x_train.fillna(0)
# todo: Mit hist-rolling-average ersetzen

In [ ]:
# Model 4 - Keras Simple Model ###############

seed = 424
tf.random.set_seed(seed)
np.random.seed(seed)
x_dim = x_train.shape[1]
y_dim = 1

def create_model():
    inputs = keras.Input(shape=(x_dim,))
    x = layers.Dense(30, activation="relu", name="d1")(inputs)
    x = layers.Dense(60, activation="relu", name="d2")(x)
    x = layers.Dense(30, activation="relu", name="d3")(x)
    outputs = layers.Dense(y_dim, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name="CNN")
    model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

    return model

clf = create_model()

In [ ]:
clf.fit(x_train.values, y_train.values, batch_size=128, epochs=1)
# %time clf.fit(x_train.values, y_train.values, batch_size=128, epochs=1)

In [ ]:
y_predictions = clf.predict(x_train.values).round().astype('int')

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_train, y_predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#%%time
for (test_df, sample_prediction_df) in iter_test:
    x_test = test_df.loc[:, test_df.columns.str.contains('feature')]
    x_test = x_test.fillna(0)
    
    if x_test.iloc[0,:].sum()==0:
        sample_prediction_df.action = 0
    else:
        y_preds = clf.predict(x_test.values).round().astype('int')
        sample_prediction_df.action = y_preds

    env.predict(sample_prediction_df)

In [ ]:
print('done')

In [ ]:
# @njit(fastmath = True)
# def utility_score_numba(date, weight, resp, action):
#     Pi = np.bincount(date, weight * resp * action)
#     t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / len(Pi))
#     u = min(max(t, 0), 6) * np.sum(Pi)
#     return u

In [ ]:
# print('Utility score train:', utility_score_numba(train['date'].values, train['weight'].values, train['resp'].values, train['action'].values))